In [1]:
import numpy as np
import pandas as pd
from scipy.stats import *
import random
import seaborn as sns

In [2]:
df = pd.read_csv(r'C:\Users\mohit\Downloads\ML Datasets\titanic.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
df.shape

(891, 12)

# Chi square test:

In [6]:
d = df.copy()

In [7]:
d['Survived'] = np.where(d['Survived']==1, "survived", "not survived")

In [8]:
ct = pd.crosstab(d['Pclass'], d['Embarked'])

In [9]:
ct

Embarked,C,Q,S
Pclass,,,
1,85,2,127
2,17,3,164
3,66,72,353


In [10]:
chi2_contingency(ct)

Chi2ContingencyResult(statistic=123.75190952951289, pvalue=8.435267819894384e-26, dof=4, expected_freq=array([[ 40.44094488,  18.53543307, 155.02362205],
       [ 34.77165354,  15.93700787, 133.29133858],
       [ 92.78740157,  42.52755906, 355.68503937]]))

In [11]:
1-chi2.cdf(260.71702016732104, df=1)

0.0

In [12]:
no_rows = len(ct.iloc[:,0])

In [13]:
no_cols = len(ct.iloc[0,:])

In [14]:
print(no_rows,no_cols)

3 3


In [15]:
freedom = (no_rows-1)*(no_cols-1)

In [16]:
freedom

4

In [17]:
expected = np.zeros((len(ct.iloc[:,0]),len(ct.iloc[0,:])))
for i in range(len(ct.iloc[:,0])):
    for j in range(len(ct.iloc[0,:])):
        expected[i][j] = np.sum(ct.iloc[i,:])*np.sum(ct.iloc[:,j])/np.sum(ct.values)

In [18]:
expected

array([[ 40.44094488,  18.53543307, 155.02362205],
       [ 34.77165354,  15.93700787, 133.29133858],
       [ 92.78740157,  42.52755906, 355.68503937]])

In [19]:
observed = ct.values

In [20]:
chi_stat = np.sum(sum([(o-e)**2/e for o,e in zip(observed,expected)]))

In [21]:
print(chi_stat)

123.75190952951287


In [22]:
chi2.ppf(q = 0.05, df = 4)

0.7107230213973239

In [23]:
1-chi2.cdf(chi_stat,df=4)

0.0

# t-test:

In [24]:
# problem statement:
# is there difference between sample mean and population mean
# alpha = 0.05

In [25]:
pop = df['Age'].dropna()

In [26]:
sample = pop.sample(25)

In [27]:
t_stat, p_val = ttest_1samp(sample, 35)

In [28]:
print('t_statistics',t_stat) 
print('p_value:', p_val)

t_statistics -1.7842283159205483
p_value: 0.0870360723490897


In [29]:
(1-t.cdf(abs(t_stat),24))*2

0.08703607234908972

In [30]:
sample_mean = np.mean(sample)
sample_std = np.std(sample, ddof=1)

In [31]:
t_stat_=(sample_mean-35)*((len(sample)-1)**0.5)/sample_std

In [32]:
t_stat_

-1.7481795834522749

In [33]:
critical_val = t.ppf(q=1-0.05/2,df=24)
print(critical_val)

2.0638985616280205


In [34]:
p_=(1-t.cdf(abs(t_stat_), df=24))*2

In [35]:
p_

0.09321591444445132

In [36]:
sample.mean()

30.36

# Confidence Interval t-test:

In [37]:
sample_mean = sample.mean()

In [38]:
alpha = 0.05
degree_freedom = len(sample)-1

In [39]:
t_critical = t.ppf(1-alpha/2, df = degree_freedom)

In [40]:
lower_interval = sample_mean-t_critical*np.std(sample, ddof=1)/(degree_freedom**0.5)

In [41]:
upper_interval = sample_mean+t_critical*np.std(sample, ddof=1)/(degree_freedom**0.5)

In [42]:
print(f'CI from t_test: {lower_interval} < CI < {upper_interval}')

CI from t_test: 24.882021984124464 < CI < 35.837978015875535


##### using scipy library: 

In [43]:
t.interval(confidence = 0.95, df=degree_freedom, loc = sample_mean, scale = sem(sample))

(24.992699615568448, 35.72730038443155)

In [44]:
sem(sample)

2.600564041382818

In [45]:
np.std(sample, ddof=1)/(len(sample)**0.5)

2.600564041382818

# two sample t-test:

In [46]:
s1 = [12, 15, 18, 16, 20, 17, 14, 22, 19, 21, 23, 18, 25, 17, 16, 24, 20, 19, 22, 18, 15, 14, 23, 16, 12, 21, 19, 17, 20, 14]
s2 = [10, 12, 14, 13, 16, 15, 11, 17, 14, 16, 18, 14, 20, 15, 14, 19, 16, 15, 17, 14, 12, 11, 18, 15, 10, 16, 15, 13, 16, 11]

In [47]:
m1 = np.mean(s1)
m2 = np.mean(s2)

In [48]:
sd1 = np.std(s1, ddof=1)
sd2 = np.std(s2, ddof=1)

In [49]:
t2_stat = (m1-m2)/np.sqrt((sd1**2/len(s1))+(sd2**2/len(s2)))

In [50]:
print(t2_stat)

4.625335930681123


In [51]:
t2_critical = t.ppf(q=1-0.05/2, df=58)
t2_critical

2.0017174830120923

In [52]:
t2_p_value = (1-t.cdf(abs(t2_stat),df=58))*2

In [53]:
print(t2_p_value)

2.1422811335103376e-05


In [54]:
ttest_ind(s1,s2)

TtestResult(statistic=4.625335930681123, pvalue=2.1422811334975257e-05, df=58.0)

# Confidence Interval z-test:

In [55]:
pop_mean = pop.mean()

In [56]:
pop_sd = np.std(pop)

In [57]:
z_sample = df[df['Survived']==1]['Age'].dropna().sample(25)

In [58]:
z_critical = norm.ppf(q=1-alpha/2)

In [59]:
pop_lower = pop_mean-z_critical*pop_sd/np.sqrt(len(z_sample))

In [60]:
pop_upper = pop_mean+z_critical*pop_sd/np.sqrt(len(z_sample))

In [61]:
print(f'CI from z-test is: {pop_lower} < CI < {pop_upper}')

CI from z-test is: 24.008824318335027 < CI < 35.38941097578262


In [62]:
norm.interval(confidence=0.95, loc = pop_mean, scale = pop_sd/np.sqrt(len(z_sample)))

(24.008824318335027, 35.38941097578262)

### z_statistic: 

In [63]:
z_stat = (np.mean(z_sample)-pop_mean)*np.sqrt(len(z_sample))/pop_sd

In [64]:
z_stat

1.357397067754765

In [65]:
z_critical

1.959963984540054

In [66]:
(1-norm.cdf(z_stat))*2

0.17465507943952274

# ANNOVA test:

#### One Way ANOVA test: 

In [67]:
A = np.array([8,10,12,8,7])
B = np.array([12,11,9,14,4])
C = np.array([18,12,16,6,8])
D = np.array([13,9,12,16,15])

In [68]:
f_oneway(A,B,C,D)

F_onewayResult(statistic=1.2820512820512822, pvalue=0.3143594700488932)

In [79]:
f.ppf(q=1-0.05,dfn=3,dfd=16)

3.238871517453585

In [81]:
1-f.cdf(1.2820512820512822, dfn = 3, dfd = 16)

0.3143594700488932

#### Two Way ANOVA test: 

In [103]:
data = {'season': ['summer', 'winter', 'monsoon'],
        'A': [36, 28, 26],
        'B': [36, 29, 28],
        'C': [21, 31, 29],
        'D': [35, 32, 29]}
df_anova = pd.DataFrame(data)

In [104]:
df_anova

,season,A,B,C,D
0,summer,36,36,21,35
1,winter,28,29,31,32
2,monsoon,26,28,29,29


In [105]:
df_melt = pd.melt(df_anova, id_vars=['season'], value_vars=['A', 'B', 'C', 'D'], var_name='salesman', value_name='sales')

In [106]:
df_melt

,season,salesman,sales
0,summer,A,36
1,winter,A,28
2,monsoon,A,26
3,summer,B,36
4,winter,B,29
5,monsoon,B,28
6,summer,C,21
7,winter,C,31
8,monsoon,C,29
9,summer,D,35


In [94]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [110]:
model = ols('sales~salesman+season', data=df_melt).fit()

In [112]:
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
salesman,3.0,42.0,14.000000,0.617647,0.628574
season,2.0,32.0,16.000000,0.705882,0.530504
Residual,6.0,136.0,22.666667,NaN,NaN
